# Deep Research tool with Azure AI Foundry (preview)

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

The **Deep Research tool** in the Azure AI Foundry Agent Service enables you to integrate a web-based research capability into your systems. The Deep Research capability is a specialized AI capability designed to perform in-depth, multi-step research using data from the public web.

> https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research

The **o3-deep-research model** and the GPT model deployments should be part of your AI Foundry project resulting in all three resources in the same Azure subscription and same region. Supported regions are **West US and Norway East.**

At its core, **Deep Research** leverages a combination of OpenAI and Microsoft technologies, including **o3-deep-research**, various GPT models, and **Bing Search Grounding**, when integrated into an agent.

- When an agent with Deep Research integration receives a research request — whether from a user or another application — it utilizes GPT-4o and GPT-4.1 to interpret the intent, fill in any missing details, and define a clear, actionable scope for the task.
- Once the task is defined, the agent activates the Bing-powered grounding tool to gather a refined selection of recent, high-quality web content.
- Following this, the o3-deep-research agent begins the research process by reasoning through the collected information. Rather than merely summarizing content, it evaluates, adapts, and synthesizes insights from multiple sources, adjusting its approach as new data emerges.
- The entire process culminates in a structured report that not only provides the answer but also includes the model’s reasoning path, source citations, and any clarifications requested during the session, as explained by Arenas.

In [1]:
#%pip install --pre azure-ai-projects
#%pip install pypandoc
#%pip install azure-identity

In [2]:
import datetime
import os
import pypandoc
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import DeepResearchTool, MessageRole, ThreadMessage
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from IPython.display import display, FileLink, Markdown
from typing import Optional

In [3]:
sys.version

'3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 17.0.0 (clang-1700.0.13.3)]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 15-Jul-2025 11:35:17


## Settings

> Only available now in "West US" and "Norway East"

In [5]:
# Foundry project
project_endpoint = "https://foundry-norway-east.services.ai.azure.com/api/projects/foundry-project-norway-east"

In [6]:
model = "gpt-4.1"  # a generic gpt model
deep_research_model = "o3-deep-research"  # the new o3 deep research model deployed in your AI Foundry

bingservice = "aqbinggrounding002"  # The Bing connection service in your AI foundry project

In [7]:
RESULTS_DIR = "documents"

os.makedirs(RESULTS_DIR, exist_ok=True)

In [8]:
now = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')

md_results_file = os.path.join(RESULTS_DIR, f"deep_research_results_{now}.md")  # The name of the markdown output file
docx_file = os.path.join(RESULTS_DIR, f"deep_research_results_{now}.docx")  # .docx outpyt

## Helper

In [9]:
def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    """
    Fetches and prints the latest response from an agent in a given thread.

    Args:
        thread_id (str): The ID of the thread to fetch the agent's response from.
        agents_client (AgentsClient): The client used to interact with the agents service.
        last_message_id (Optional[str], optional): The ID of the last message that was processed. Defaults to None.

    Returns:
        Optional[str]: The ID of the latest message if there is new content, otherwise returns the last_message_id.
    """
    response = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id  # No new content

    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))

    for ann in response.url_citation_annotations:
        print(
            f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})"
        )

    return response.id

In [10]:
def create_research_summary(message: ThreadMessage,
                            filepath: str = md_results_file) -> None:
    """
    Creates a research summary from the provided message and writes it to a file.

    Args:
        message (ThreadMessage): The message containing the content for the research summary.
        filepath (str, optional): The path to the file where the research summary will be written in a markdown format.

    Returns:
        None
    """
    if not message:
        print("Error: No message content provided")
        return

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join(
            [t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- [{title}]({url})\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")

## Project & tool definitions

In [11]:
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

conn_id = project_client.connections.get(name=bingservice).id

In [12]:
deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=deep_research_model,
)

## Example

In [13]:
prompt = "Give me the latest research into magentic-ui from Microsoft's autogen over the last year. Do not ask questions"

In [14]:
start = time.time()

with project_client:
    with project_client.agents as agents_client:
        agent = agents_client.create_agent(
            model=model,
            name="deep-research-agent",
            instructions=
            "You are a helpful agent that assists in researching scientific topics.",
            tools=deep_research_tool.definitions,
        )

        print(f"🎉 Created agent, ID: {agent.id}")

        thread = agents_client.threads.create()
        print(f"🧵 Created thread, ID: {thread.id}")

        # Create message to thread
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=(prompt),
        )
        print(f"📩 Created message, ID: {message.id}")
        print(
            f"⏳ Start processing the message... this may take a few minutes to finish. Be patient!"
        )

        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
        last_message_id = None

        while run.status in ("queued", "in_progress"):
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

            last_message_id = fetch_and_print_new_agent_response(
                thread_id=thread.id,
                agents_client=agents_client,
                last_message_id=last_message_id,
            )
            print(f"🔄 Run status: {run.status}")

        print(f"✅ Run finished with status: {run.status}, ID: {run.id}")

        if run.status == "failed":
            print(f"❌ Run failed: {run.last_error}")

        final_message = agents_client.messages.get_last_message_by_role(
            thread_id=thread.id, role=MessageRole.AGENT)
        if final_message:
            create_research_summary(final_message)

        # Clean-up and delete the agent once the run is finished.
        #agents_client.delete_agent(agent.id)
        #print("🗑️ Deleted agent")

elapsed = time.time() - start
minutes, seconds = divmod(elapsed, 60)
print(f"\nElapsed time = {minutes:.0f} minutes and {seconds:.0f} seconds")

🎉 Created agent, ID: asst_qR2G6ZR1f4CWHN5FHeAUER6c
🧵 Created thread, ID: thread_0tOPCGBVLfxhgkx21YJhZKqI
📩 Created message, ID: msg_Qrjvkt5etXhxEdaBrSKByBTj
⏳ Start processing the message... this may take a few minutes to finish. Be patient!

Agent response:
I'll gather the most recent research from the last year related to 'magentic-ui' within Microsoft's AutoGen, including advancements, publications, and main findings. I'll share a structured summary with clear sources.

Title: Latest Research on Magentic-UI in Microsoft AutoGen

 Starting deep research... 

🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS
🔄 Run status: RunStatus.IN_PROGRESS

Agent response:
cot_summary: **Clarifying terminology**

I’m digging into 'magentic-ui' from Microsoft's AutoGen. It could be an internal tool o

## Results

In [15]:
!ls -lh $md_results_file

-rw-r--r--@ 1 arturoquiroga  staff    21K Jul 15 11:46 documents/deep_research_results_15Jul2025_113517.md


In [16]:
with open(md_results_file, 'r', encoding='utf-8') as file:
    markdown_content = file.read()
    display(Markdown(markdown_content))

Final Report:
# Magentic-UI: A Human-Centered AI Web Agent on Microsoft AutoGen

## Background: AutoGen Framework and Magentic-One  
**AutoGen** is Microsoft’s open-source framework for building multi-agent AI systems. In late 2024, Microsoft Research introduced **Magentic-One**, a generalist multi-agent system built on AutoGen, designed to tackle complex, open-ended web and file-based tasks【33:1†source】,【33:1†source】. Magentic-One uses a lead **Orchestrator** agent that plans and coordinates several specialized agents (for web browsing, file handling, coding, etc.) to accomplish multi-step goals【33:1†source】,【33:2†source】. It achieved performance on par with state-of-the-art agentic systems across multiple benchmarks – for example, matching top methods on challenging web-interaction tasks like GAIA, AssistantBench, and WebArena – all without custom-training new skills【33:3†source】,【33:1†source】. The system’s modular design allows agents to be added or removed as needed without reworking the entire architecture, making it highly extensible【33:3†source】. Microsoft open-sourced Magentic-One on the AutoGen platform (MIT-licensed) and provided a **technical report** detailing its architecture and results【33:3†source】. To facilitate rigorous evaluation of such AI agents, the team also released **AutoGenBench**, a benchmarking tool with controls for repeated trials and isolated execution to minimize side-effects during testing【33:3†source】.

## Introducing Magentic-UI: A Human-Centered Web Agent  
Building on the Magentic-One foundation, Microsoft Research announced **Magentic-UI** in May 2025 as an experimental, human-centered AI web agent. Magentic-UI is an open-source prototype interface that collaborates with users in real time to automate complex web-based tasks while keeping the user **in control**, . Unlike fully autonomous agents that operate opaquely, Magentic-UI works *side-by-side* with the user in a web browser, offering transparency into its actions and plans . Users and the AI co-plan the task steps, and the system actively seeks user guidance or approval at decision points, rather than executing unchecked actions. This design addresses a key limitation of earlier agents: the lack of visibility into what the AI is doing and inability for users to intervene mid-task【33:4†source】,【33:4†source】. Magentic-UI is released under the MIT License and is available on GitHub, and it’s also integrated into Azure AI **Foundry** (an innovation hub for experimental AI), making it easy for developers and researchers to try out. In essence, Magentic-UI represents a shift from “autonomous” web automation towards a **human-in-the-loop** approach, prioritizing user oversight, safety, and collaboration【33:5†source】.

## Key Features of Magentic-UI  
Magentic-UI introduces several new features and interaction mechanisms to ensure a **collaborative and transparent** user experience:

- **Co-Planning:** The user and agent collaboratively draft a step-by-step plan for the task before execution. Magentic-UI proposes an initial natural-language plan, which the user can review and modify using a plan editor (adding, removing, or editing steps). The plan must be approved by the user, ensuring the AI’s strategy aligns with the user’s intent before any action is taken. This up-front planning process may add a bit of initial overhead, but it often saves time by preventing mistakes and improves the chances of success.

- **Co-Tasking:** The user remains in the loop during task execution. Magentic-UI provides real-time visibility into its actions and allows the user to **pause or intervene** at any point, . A live browser view is displayed alongside the plan, showing the web pages and interactions as the AI navigates them. The user can directly take control (e.g. click or type in the live browser panel) or give feedback via chat to guide the AI if it goes off track. The agent is also capable of asking the user for clarification or help when it encounters ambiguity or obstacles. This two-way collaboration ensures the AI doesn’t stray far from the user’s goals.

- **Action Guards (Safety Approvals):** Magentic-UI never performs potentially risky or irreversible actions without asking the user first. It implements *action guards* that pause the agent whenever a sensitive step is reached – for instance, clicking a “Submit” or “Buy” button, deleting data, or navigating to an untrusted site. The user must explicitly approve these guarded actions before the agent continues. Users can configure how often approvals are required or define specific allow-lists for websites and actions【33:5†source】,. In other words, if Magentic-UI is about to do something significant like finalizing a purchase or closing a tab, it will **“ask first”** instead of acting unilaterally【33:5†source】. This mechanism gives users fine-grained control to prevent unwanted outcomes.

- **Plan Learning and Reuse:** The system can **learn from experience** by saving successful task plans and reusing them in the future. After completing a task, Magentic-UI stores the verified plan in a **plan gallery**. When a similar task is requested later, it can retrieve a past plan (or parts of it) as a starting point instead of planning from scratch. This capability allows repeat or periodic tasks to be completed much faster on subsequent runs. In fact, reusing a previously learned plan can reduce the execution time or latency of a task by up to **3×** compared to generating a new plan from zero. Over time, Magentic-UI “gets better” at tasks the more it does them, since it refines its strategies and avoids past mistakes.

- **Parallel Task Execution:** To boost productivity, Magentic-UI supports running multiple tasks in parallel within separate sessions. The interface includes a session manager (on the sidebar) where users can launch new task sessions or switch between ongoing tasks. Visual status indicators show each session’s state – for example, whether the agent is waiting for user input (🔴), actively working (↺), or finished (✅),. This way, a user can supervise several automated tasks concurrently. For instance, one session might be filling out a form while another scrapes data from a website, and the user can hop between them as needed. Parallel execution, combined with the session status alerts, ensures efficiency without losing oversight.

## System Architecture and Technical Improvements  
Magentic-UI is built on the same multi-agent architecture as Magentic-One, with a team of specialized AI agents coordinated by a central orchestrator. The major components of the system include:  

- **Orchestrator:** The lead agent (powered by a large language model, e.g. GPT-4) that decomposes the user’s request into a plan and oversees its execution. The Orchestrator decides which steps to delegate to which agent (or back to the user) and monitors overall progress. It maintains a *Task Ledger* of facts, goals, and the current plan, and a *Progress Ledger* to track execution status【33:2†source】,【33:2†source】. If a step fails or conditions change, the Orchestrator can even revise the plan on the fly (with user permission) to recover from errors. This dynamic planning loop allows the system to adapt to unexpected situations while keeping the user informed.  

- **WebSurfer:** An LLM-driven web-browsing agent that controls a real browser. It can navigate to URLs, click links or buttons, fill in text fields, scroll pages, and interact with web content as needed【33:2†source】. In Magentic-UI, the WebSurfer’s capabilities have been **significantly enhanced** compared to the original Magentic-One version. It can manage multiple browser tabs, select options from drop-down menus, handle file uploads, and even perform multimodal queries (interpreting text and images on a page). These improvements enable the agent to handle more complex web interfaces and tasks than before. The WebSurfer reports back the results of each action (e.g. the new page content or state) to the Orchestrator, which is displayed live in the UI.  

- **Coder:** A coding agent specialized in writing and executing code. When a task requires data processing, computation, or automation beyond simple browsing, the Orchestrator calls on the Coder agent. The Coder can generate Python or shell scripts to perform subtasks (for example, parsing data or performing calculations). Importantly, Magentic-UI executes all code in a **secure Docker container**, isolating these operations from the user’s actual system. The Docker sandbox prevents any harmful commands from affecting the host machine and ensures that the agent cannot access the user’s local files or credentials unless explicitly allowed. After running the code, the Coder returns the results or output back to the Orchestrator.  

- **FileSurfer:** A file-handling agent that can navigate directories and read or convert files. If the task involves analyzing a local document, searching within files, or extracting information, the FileSurfer agent is tasked with those steps【33:2†source】,. It uses tools (like the MarkItDown toolkit) to convert files into readable text (e.g. converting PDFs or images to Markdown) and can answer questions about file contents. Like the Coder, the FileSurfer operates within a sandboxed environment (via Docker) to ensure it only accesses permitted files. This agent enables Magentic-UI to handle tasks that blend web browsing with local data processing – for example, downloading a report from the web and then summarizing it.  

- **UserProxy:** A special agent that represents the human user within the agent team. The UserProxy acts as a placeholder for the cases where part of the task is explicitly assigned back to the user. For instance, the Orchestrator may determine that a certain step (like deciding on a preference or providing a piece of personal information) is better handled by the user; it will delegate that step to the UserProxy agent, which essentially pauses and awaits the real user’s input,. In practice, this means Magentic-UI knows when to *ask* the user to do something rather than attempting it blindly. The inclusion of a UserProxy ensures that the human can be seamlessly integrated into the workflow as another “agent” when needed, under the Orchestrator’s coordination.

**Interface and Controls:** The Magentic-UI front-end ties these agents together in a cohesive interface. The display is split into a **dual-panel view**: one side shows the agent’s plan, step-by-step progress, and any questions for the user, while the other side shows a live browser window under the WebSurfer’s control,. As the Orchestrator and agents work through the plan, every intermediate action and result is logged visibly. A progress bar at the top visualizes how close the task is to completion. Users interact through a chat prompt and the plan editor on the left panel, and they can also directly manipulate the live browser panel on the right if needed (e.g. to demonstrate an action). This transparent UI lets users literally *see* what the AI is doing at each moment, which builds trust and allows timely intervention if something looks off-track,.

**Safety Mechanisms:** Magentic-UI’s design places a strong emphasis on safety and preventing unintended side-effects. All agent actions (web browsing and code execution) run within **sandboxed Docker containers**, providing a protective barrier between the AI agents and the user’s actual environment【33:5†source】. This means the AI cannot make system-level changes or access sensitive data on the host machine. For example, any script executed by the Coder agent runs in isolation, and any browser session run by WebSurfer is contained (with no saved cookies or credentials from the user unless provided). Additionally, users can define allowed domains or apply filters to constrain where the WebSurfer agent is permitted to navigate【33:5†source】. Combined with the earlier-mentioned *Action Guards* (which require manual confirmation for high-risk steps), these measures form a multi-layered defense. In fact, Microsoft conducted **red-team evaluations** on Magentic-UI’s agent system to probe its robustness against malicious scenarios. The tests included attempts at phishing (tricking the agent into revealing sensitive info or visiting harmful sites) and prompt injection attacks. The results were encouraging – Magentic-UI either sought user clarification or outright blocked the unsafe actions in those cases, rather than blindly executing them. This demonstrates that the system’s safeguards and human-in-the-loop checks can effectively mitigate many potential risks. Magentic-UI also runs by default with no persistent login credentials in the browser, ensuring that the agent cannot accidentally leverage the user’s authenticated sessions on websites. All these technical improvements (sandboxing, allow-lists, approval gates, etc.) are aimed at making the agent **transparent, secure, and controllable** in real-world use,.

## Performance and Notable Findings  
Over the past year, Microsoft has evaluated Magentic-UI’s performance both as an autonomous agent and with human-in-the-loop assistance. The underlying multi-agent system already proved capable on complex tasks: **Magentic-One** (without user input) demonstrated competitive results on several benchmarks of agent performance【33:1†source】. For example, it achieved strong success rates on the **GAIA** challenge (a test of general AI assistants requiring reasoning, tool use, and web interaction) and on **WebArena/WebGames** (interactive web browsing tasks), coming close to prior state-of-the-art systems【33:3†source】,. These results were attained with Magentic-One’s generalist architecture *without* needing specialized training or tuning for each task, highlighting the power of the modular multi-agent approach【33:1†source】.

More interestingly, Magentic-UI’s human-centered features have led to **significant improvements** in task completion when even minimal user guidance is provided. In an experiment on the GAIA benchmark, allowing the agent to ask for occasional help or confirmation from a (simulated) user boosted the task success rate from about **30.3% to 51.9%**, a relative increase of **71%** in completion rate. In other words, by inserting a bit of human feedback into the loop, the agent solved many more tasks correctly than it could autonomously. Notably, Magentic-UI achieved this boost with only **light user involvement** – the system needed to request help in only roughly **10%** of the tasks, averaging just **1.1** user interactions per task in that experiment. This suggests that a small amount of timely human oversight can have a outsized positive impact on an agent’s reliability. As one report put it, *“in some scenarios, task completion went up by 71% just by including light human feedback”*【33:5†source】. These findings validate Magentic-UI’s design philosophy: rather than pursuing full autonomy at any cost, keeping a human in the loop for guidance and error correction dramatically improves outcomes on complex tasks while still maintaining efficiency.

Another notable benefit observed is the impact of **plan reuse and learning** on efficiency. As mentioned, Magentic-UI can save successful workflows and retrieve them later. In tests, this **plan gallery** mechanism reduced the time to complete repeat tasks by up to threefold, since the agent could skip recalculating steps and immediately deploy a proven plan. For users, this means the agent becomes faster and more effective with each similar task it tackles – a form of cumulative learning. Additionally, Magentic-UI performed well even with **different AI models** powering it. The system is model-agnostic; while the default uses a version of GPT-4 for the Orchestrator, experiments have shown it can incorporate other language models to balance cost and performance【33:2†source】. This flexibility implies researchers or developers can plug in newer or domain-specific models into the agent roles to further improve certain capabilities (for instance, using a specialized vision model for the WebSurfer’s image analysis or a smaller, faster model for the Coder if real-time response is crucial)【33:2†source】.

Crucially, the introduction of Magentic-UI did not reveal major new failure modes beyond those already present in agentic systems, and the added oversight actually helps catch issues. The transparent interface allowed testers to catch and correct agent mistakes on the fly, preventing the kind of hidden errors that fully-autonomous agents might carry through unnoticed. The internal evaluations, including the red-team tests described earlier, indicate that Magentic-UI’s layered safety approach is effective: the system **passed phishing and prompt-injection tests** by either refusing the malicious action or deferring to the user. This is a promising result for the safe deployment of such agents. Of course, Magentic-UI is not yet infallible – there were still tasks it could not complete and errors it could not overcome – but the overall trend shows that combining AI automation with human oversight yields a more robust and trustworthy performance.

## Conclusion and Outlook  
Magentic-UI represents a significant step forward in the evolution of “agentic” AI systems, demonstrating how **human-AI collaboration** can improve both the effectiveness and safety of AI assistants. By keeping the user in the loop, the system addresses long-standing concerns about transparency and loss of control in autonomous AI. Rather than aiming to replace the user, Magentic-UI is designed to **empower** the user – handling the busywork of web-based tasks while continually soliciting user guidance for critical decisions. Early research indicates that this approach can substantially boost task success rates (as seen with the 71% improvement on GAIA) without imposing a heavy burden on the user,. The open-source release of Magentic-UI (and its underlying Magentic-One framework) is also notable – it invites the broader research and developer community to experiment with the system, reproduce results, and build upon its human-centered design【33:5†source】,. Microsoft’s team has highlighted that Magentic-UI is still a prototype and **far from human-level performance** on many tasks, and there are open challenges to tackle around reliability and potential misuse【33:1†source】. As AI agents become more powerful, ensuring they remain **helpful, aligned, and safe** will be crucial【33:1†source】. Magentic-UI provides a valuable platform to explore these questions in a grounded way. 

In summary, over the past year Microsoft’s AutoGen project has evolved from the multi-agent capabilities of Magentic-One to the human-centric interface of Magentic-UI – adding transparency, user feedback loops, and safety features to create a more **trustworthy AI assistant for the web**. This research prototype has demonstrated encouraging results in combining the strengths of AI and human problem-solving. It lays a strong foundation for future intelligent assistants that can handle complex tasks *with* us rather than for us, ensuring that we as users maintain clarity and control. The work on Magentic-UI not only showcases new features and technical improvements in agent design, but also offers insights and tools (like AutoGenBench and the open-source codebase) for the wider AI community to advance the state of the art in cooperative, human-aligned AI systems. The coming years will likely build on these ideas, bringing us closer to AI agents that are both highly capable and **deeply collaborative** with their human partners. 

**Sources:** The information in this report is derived from Microsoft Research publications, technical documentation, and reputable AI industry analyses. Key references include the Microsoft Research blog announcements of *Magentic-One* (Nov 2024) and *Magentic-UI* (May 2025)【33:1†source】,, the Magentic-One technical report on arXiv【33:3†source】, the official Magentic-UI GitHub documentation,, and summaries by trusted tech outlets such as MarkTechPost and BetaNews that covered Magentic-UI’s features and experimental results,【33:5†source】. These sources provide detailed accounts of the system’s design, capabilities, and the latest research findings supporting the above summary.

## References
- [Magentic-One: A Generalist Multi-Agent System for Solving Complex Tasks ...](https://www.microsoft.com/en-us/research/articles/magentic-one-a-generalist-multi-agent-system-for-solving-complex-tasks/)
- [Microsoft AI Introduces Magentic-UI: An Open-Source Agent Prototype ...](https://www.marktechpost.com/2025/05/22/microsoft-ai-introduces-magentic-ui-an-open-source-agent-prototype-that-works-with-people-to-complete-complex-tasks-that-require-multi-step-planning-and-browser-use/)
- [Microsoft Magentic-UI is an open source AI tool that lets humans stay ...](https://betanews.com/2025/05/19/microsoft-magentic-ui-is-an-open-source-ai-tool-that-lets-humans-stay-in-control/)
- [Using Magentic-UI, Microsoft’s Open-Source Human-centric web agent](https://www.thewindowsclub.com/use-microsofts-open-source-human-centric-web-agent-magentic-ui)


### Exporting results into a docx file

In [17]:
pypandoc.convert_file(md_results_file, 'docx', outputfile=docx_file)
print(f"{md_results_file} has been converted to {docx_file}\n")

!ls -lh $docx_file

OSError: No pandoc was found: either install pandoc and add it
to your PATH or or call pypandoc.download_pandoc(...) or
install pypandoc wheels with included pandoc.

In [ ]:
link = FileLink(path=docx_file)
link